**[SQL Course Home Page](https://www.kaggle.com/learn/SQL)**

---


# Intro

You are getting to the point where you can own an analysis from beginning to end. So you'll do more data exploration in this exercise than you've done before.  Before you get started, run the following set-up code as usual. Notice that your `BigQueryHelper` object is called `chicago_taxi_helper`.

In [1]:
# Set up feedack system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex5 import *

# import package with helper functions 
import bq_helper

# create a helper object for this dataset
chicago_taxi_helper = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                               dataset_name="chicago_taxi_trips")

Using Kaggle's public dataset BigQuery integration.
Using Kaggle's public dataset BigQuery integration.


# Questions

This dataset contains a table with the information about taxi trips in the city of Chicago. You are curious how much slower traffic moves when traffic volume is high. This involves a few steps.

# 1) Find the Data
Before you can access the data, you need to find the table name with the data.

- Hint: Tab completion is helpful whenever you can't remember a command. Type `chicago_taxi_helper.` and then hit the tab key. Don't forget the period before hitting tab.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "crypto_bitcoin" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# print list of tables
for table in tables:
    print(table.table_id)

Using Kaggle's public dataset BigQuery integration.
taxi_trips


In [3]:
# write the table name as a string below
table_name = 'taxi_trips'

q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# q_1.solution()

# 2) Peek at the data

Use `chicago_taxi_helper.head` to peak at the top few rows of the data. Inspect the data and see if any data quality are immediately obvious. After deciding whether you see any important issues, look at `q_2.solution()`.

In [4]:
# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "taxi_trips" table
client.list_rows(table, max_results=5).to_dataframe()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,8942f4c92b492ef57eec0777046211e300f47c99,9eb860ce5cb7ff8e35eaf83b4a6d5a5f97710448b10fd7...,2014-09-21 14:00:00+00:00,2014-09-21 14:15:00+00:00,600,2.6,None,None,None,None,9.05,0.0,0.0,0.0,9.05,Cash,None,None,None,None,None,None,None
1,6847a885605e1a98a7e97cb7a0922b5d5dc4d576,9eb860ce5cb7ff8e35eaf83b4a6d5a5f97710448b10fd7...,2014-09-21 14:45:00+00:00,2014-09-21 14:45:00+00:00,480,1.7,None,None,None,None,7.45,0.0,0.0,0.0,7.45,Cash,None,None,None,None,None,None,None
2,7357a709641e20156839ead21822f4e07d8e50f5,9eb860ce5cb7ff8e35eaf83b4a6d5a5f97710448b10fd7...,2014-09-21 14:45:00+00:00,2014-09-21 15:00:00+00:00,480,0.8,None,None,None,None,6.05,0.0,0.0,0.0,6.05,Cash,None,None,None,None,None,None,None
3,793b87591b2afd879eed1fd871547a25c7a917e5,9eb860ce5cb7ff8e35eaf83b4a6d5a5f97710448b10fd7...,2014-09-21 14:30:00+00:00,2014-09-21 14:30:00+00:00,660,2.2,None,None,None,None,8.65,0.0,0.0,0.0,8.65,Cash,None,None,None,None,None,None,None
4,8858a7c0141d5a9c298b450cd6fc4b91f1ee1ab2,9eb860ce5cb7ff8e35eaf83b4a6d5a5f97710448b10fd7...,2014-09-21 12:45:00+00:00,2014-09-21 13:00:00+00:00,360,1.1,None,None,None,None,6.05,0.0,0.0,1.0,7.05,Cash,None,None,None,None,None,None,None


In [ ]:
chicago_taxi_helper.head 

In [ ]:
q_2.solution()

# 3) Determine when this data is from

If the data is sufficiently old, we might be careful before assuming the data is still relevant to traffic patterns today. Write a query that counts the number of trips in each year.  

Your results should have two columns
1. The year of the trips, which is called `year`
2. The number of trips in that year, called `num_trips`.

Hints:
- When using `GROUP BY` and `ORDER BY`, you should refer to the columns by the alias `year` that you set at the top of the **SELECT** query.
- The sql code to select the Year from `trip_start_timestamp` is <code>SELECT **EXTRACT(YEAR FROM trip_start_timestamp)**</code>
- The FROM field can be a little tricky until you are used to it.  The format is
    1. A backick (the symbol \`)
    2. The project name. You can find this in the line where we created the BigQueryHelper object. In this case it is `bigquery-public-data`
    3. A period
    4. The dataset name. Also in the line where we created the BigQueryHelper object. In this case, it is `chicago_taxi_trips`
    5. A period
    6. The table name, which you have already used as your answer in part 1.



In [5]:
# Query to select the number of rides per year, sorted by year
rides_per_year_query = """
                        SELECT EXTRACT(YEAR FROM trip_start_timestamp) AS year,
                                     COUNT(1) AS num_trips
                        FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                        GROUP BY year
                        ORDER BY year
                        """
        
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_year_query_job = client.query(rides_per_year_query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
rides_per_year_result = rides_per_year_query_job.to_dataframe()

print(rides_per_year_result)
q_3.check()

   year  num_trips
0  2013   27217716
1  2014   37395436
2  2015   32385875
3  2016   31759339
4  2017   24988003
5  2018   20732088
6  2019    5500519


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# q_3.hint()
# q_3.solution()

# 4) Dive Slightly Deeper

It's odd that 2017 had so few rides. You should wonder whether it was systematic under-reporting throughout the year, or whether some months are missing.  Copy the query you used above in `rides_per_year_query` into the cell below for `rides_per_month_query`.  Then modify it in two ways
1. Use a where clause to limit our query to data from 2017
2. Modify the extract statement to extract the month rather than the year


In [6]:
rides_per_month_query = """                       
                        SELECT EXTRACT(MONTH FROM trip_start_timestamp) AS month,
                            COUNT(1) AS num_trips
                        FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                        WHERE EXTRACT(YEAR FROM trip_start_timestamp) = 2017
                        GROUP BY month
                        ORDER BY month
                        """
        
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_month_query_job = client.query(rides_per_month_query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
rides_per_month_result = rides_per_month_query_job.to_dataframe()

print(rides_per_month_result)
q_4.check()

    month  num_trips
0       1    1972071
1       2    1909802
2       3    2362105
3       4    2194702
4       5    2323386
5       6    2324472
6       7    2054299
7       8    2079861
8       9    1950631
9      10    2141197
10     11    1907997
11     12    1767480


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
q_4.hint()
#q_4.solution()

# 5) Write the Query

Let's step up the sophistication of your queries.

Write a query
- A column called `hour_of_day` and sort by this, which holds the result of extracting the hour from `trip_start_timestamp`.
- The count of total number of trips in each hour of the day (e.g. how many trips occurred between 6AM and 7AM, independent of which day it occurred on). Call this `num_trips`
- The average speed, measured in miles per hour, for trips that started in that hour of the day.  Average speed in miles per hour is calculated as `3600 * SUM(trip_miles) / SUM(trip_seconds)`. Call this column `avg_mph`. The value 3600 is used to convert from seconds to hours.

For 2017, we're missing August and everything after. So restrict your query to data meeting the following criteria
- a `trip_start_timestamp` between **2017-01-01** and **2017-07-01**
- `trip_seconds` > 0 and `trip_miles` > 0


You will use a CTE to select just the relevant rides.  Because this dataset is very big, this CTE should select only the columns you'll need to create the final output (though you won't actually create those in the CTE. Instead you'll create those in the later `SELECT` statement.)

This is a much harder query than anything you've written so far.  Good luck!


In [12]:
# Query to assess the average speed per hour of day in H1 of 2017
speeds_query = """
                WITH RelevantRides AS
                    (
                    SELECT trip_start_timestamp,
                        trip_miles,
                        trip_seconds
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE trip_start_timestamp > '2017-01-01' AND 
                            trip_start_timestamp < '2017-07-01' AND
                            trip_seconds > 0 AND
                            trip_miles > 0
                    )                      
                SELECT EXTRACT(HOUR FROM trip_start_timestamp) AS hour_of_day,
                        3600 * SUM(trip_miles) / SUM(trip_seconds) AS avg_mph,
                        COUNT(1) AS num_trips
                FROM RelevantRides
                GROUP BY hour_of_day
                ORDER BY hour_of_day
                        """
        
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
speeds_query_job = client.query(speeds_query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
speeds_result = speeds_query_job.to_dataframe()

# Set high max_gb_scanned because this query looks at more data
speeds_result = chicago_taxi_helper.query_to_pandas_safe(speeds_query, max_gb_scanned=20)

print(speeds_result)
q_5.check()

    hour_of_day    avg_mph  num_trips
0             0  20.230524     319339
1             1  18.937621     266529
2             2  18.777070     210147
3             3  20.158048     159668
4             4  26.736014     122183
5             5  30.769172     119312
6             6  24.588313     182738
7             7  17.735967     358406
8             8  15.079892     541775
9             9  16.543882     565548
10           10  18.539614     525120
11           11  18.928379     594603
12           12  17.838745     622324
13           13  17.671089     630181
14           14  16.974239     622465
15           15  15.688418     640430
16           16  14.283888     701435
17           17  12.462955     756627
18           18  13.646810     768251
19           19  16.642882     701064
20           20  19.536777     598614
21           21  20.433874     552726
22           22  19.531374     501095
23           23  19.877046     399587


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
#q_5.hint()
q_5.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT EXTRACT(HOUR FROM trip_start_timestamp) AS hour_of_day, 
                          trip_miles, 
                          trip_seconds
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE trip_start_timestamp > '2017-01-01' AND 
                         trip_start_timestamp < '2017-07-01' AND 
                         trip_seconds > 0 AND 
                         trip_miles > 0
               )
               SELECT hour_of_day, 
                      COUNT(1) AS num_trips, 
                      3600 * SUM(trip_miles) / SUM(trip_seconds) AS avg_mph
               FROM RelevantRides
               GROUP BY hour_of_day
               ORDER BY hour_of_day
               """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
speeds_query_job = client.query(speeds_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
speeds_result = speeds_query_job.to_dataframe()

# View results
print(speeds_result)

```

That's a hard query. If you made good progress towards the solution, congratulations!

# 6. Ponder the Results
Something is wrong with either the raw data or our last query. What fact about the raw data doesn't seem right?

If you can identify the problem, how would you look at the raw data to verify that the problem is in the raw data and not just in your results? Check your answer below

In [ ]:
# very high value of speed at 5:00 ?
# maybe check the number of values present for this time : maybe (count), and also max values

In [9]:
q_6.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
The results show rides with hours 1-12. But there should be results in the afternoon (hours 13-24).

Perhaps the raw data has lost the distinction between AM and PM values.

You can review 200 rows of the raw data with the commands: 
```python
# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "taxi_trips" table
client.list_rows(table, max_results=200).to_dataframe()
```

You'll see that the timestamps are all in the AM hours (hours are less than or equal to 12.) 

At first you might worry that the data is coming back sorted by time, but the variety of dates suggests that's not the case. 
Part of data science is tracking down exactly this type of problem. If you were in an organization working on this, you could show the evidence you've just collected (e.g. the breakdown of trips by hour) to someone responsible for collecting the data, and help them debug the data collection and storage process.


In [10]:
# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "taxi_trips" table
client.list_rows(table, max_results=200).to_dataframe()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,8942f4c92b492ef57eec0777046211e300f47c99,9eb860ce5cb7ff8e35eaf83b4a6d5a5f97710448b10fd7...,2014-09-21 14:00:00+00:00,2014-09-21 14:15:00+00:00,600,2.6,None,None,None,None,9.05,0.00,0.0,0.0,9.05,Cash,None,None,None,None,None,None,None
1,6847a885605e1a98a7e97cb7a0922b5d5dc4d576,9eb860ce5cb7ff8e35eaf83b4a6d5a5f97710448b10fd7...,2014-09-21 14:45:00+00:00,2014-09-21 14:45:00+00:00,480,1.7,None,None,None,None,7.45,0.00,0.0,0.0,7.45,Cash,None,None,None,None,None,None,None
2,7357a709641e20156839ead21822f4e07d8e50f5,9eb860ce5cb7ff8e35eaf83b4a6d5a5f97710448b10fd7...,2014-09-21 14:45:00+00:00,2014-09-21 15:00:00+00:00,480,0.8,None,None,None,None,6.05,0.00,0.0,0.0,6.05,Cash,None,None,None,None,None,None,None
3,793b87591b2afd879eed1fd871547a25c7a917e5,9eb860ce5cb7ff8e35eaf83b4a6d5a5f97710448b10fd7...,2014-09-21 14:30:00+00:00,2014-09-21 14:30:00+00:00,660,2.2,None,None,None,None,8.65,0.00,0.0,0.0,8.65,Cash,None,None,None,None,None,None,None
4,8858a7c0141d5a9c298b450cd6fc4b91f1ee1ab2,9eb860ce5cb7ff8e35eaf83b4a6d5a5f97710448b10fd7...,2014-09-21 12:45:00+00:00,2014-09-21 13:00:00+00:00,360,1.1,None,None,None,None,6.05,0.00,0.0,1.0,7.05,Cash,None,None,None,None,None,None,None
5,17a8ada43d464623507fc85e5141a89dcda38cbf,9eb860ce5cb7ff8e35eaf83b4a6d5a5f97710448b10fd7...,2014-09-21 12:30:00+00:00,2014-09-21 12:45:00+00:00,420,1.1,None,None,None,None,6.25,2.00,0.0,0.0,8.25,Credit Card,None,None,None,None,None,None,None
6,78bf84126ec898888ba50aba0d19b63d2af02434,9eb860ce5cb7ff8e35eaf83b4a6d5a5f97710448b10fd7...,2014-09-21 12:15:00+00:00,2014-09-21 12:15:00+00:00,420,2.4,None,None,None,None,8.05,0.00,0.0,0.0,8.05,Cash,None,None,None,None,None,None,None
7,6c88853f4cbd027c6619f926d74890ad2528dbf3,9eb860ce5cb7ff8e35eaf83b4a6d5a5f97710448b10fd7...,2014-09-21 10:00:00+00:00,2014-09-21 10:15:00+00:00,1380,17.5,None,None,None,None,35.25,7.65,0.0,3.0,45.90,Credit Card,None,None,None,None,None,None,None
8,8a97b6d6aca79608daa0f244b3e9c1ace8ef3939,9eb860ce5cb7ff8e35eaf83b4a6d5a5f97710448b10fd7...,2014-09-21 09:00:00+00:00,2014-09-21 09:15:00+00:00,1200,17.3,None,None,None,None,34.65,2.50,0.0,3.0,40.15,Credit Card,None,None,None,None,None,None,None
9,f78998d8ddc600198142db69f8b787f580259776,9eb860ce5cb7ff8e35eaf83b4a6d5a5f97710448b10fd7...,2014-09-21 08:00:00+00:00,2014-09-21 08:15:00+00:00,1500,17.7,None,None,None,None,36.45,9.98,0.0,3.5,49.93,Credit Card,None,None,None,None,None,None,None


# Keep Going

You can write very complex queries now with a single data source. But nothing expands the horizons of SQL as much as the ability to combine or **JOIN** tables.

**[Click here](https://www.kaggle.com/dansbecker/joining-data)** to start the last lesson in SQL course.


---
**[SQL Course Home Page](https://www.kaggle.com/learn/SQL)**

